## basic test

In [1]:
import sys
sys.path.append('../build')
import libry as ry
import numpy as np
import time

In [2]:
#-- Add REAL WORLD configuration and camera
RealWorld = ry.Config()
RealWorld.addFile("../scenarios/check.g")

#obj1 = RealWorld.frame("obj1")
#obj1.setColor([1.,.0,.0])

S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")



In [3]:
C = ry.Config()
C.addFile('../scenarios/check.g')
V = ry.ConfigurationViewer()
V.setConfiguration(C)
cameraFrame = C.frame("camera")

In [4]:
# percept = C.addFrame("percept")
# percept.setQuaternion([1,0,0,0])
# percept.setShape(ry.ST.sphere, [.03])
# percept.setColor([0,1,0])
# percept.setPosition([0,0,2.])

In [5]:
V.recopyMeshes(C)
V.setConfiguration(C)  

In [6]:
obj = RealWorld.frame("tire")
obj.setContact(1)
obj.getPosition()

car = RealWorld.frame("car")
#car.setContact(1)

# obj.setColor([0.,1.,.0])
# obj.setPosition([0., 0, 2.])

In [7]:
print(obj.info())
print(obj.getQuaternion())

{'name': 'tire', 'ID': 164, 'shape': 'cylinder', 'size': [0.15, 0.3], 'color': [0.0, 0.0, 0.0], 'contact': 1, 'mass': 42.4115008234622, 'X': [2.0, -1.1, 0.73, -0.741170883, -0.000267996453, 1.97769841e-05, 0.671316564]}
[-7.41170883e-01 -2.67996453e-04  1.97769841e-05  6.71316564e-01]


In [8]:
def move_to_position(gipper, position):
    komo = C.komo_path(1.,1, tau, True) 
    komo.clearObjectives()
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
    komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
    komo.addObjective([1.], ry.FS.position, [gipper], ry.OT.sos, [1e3], target=position)
    komo.addObjective([], ry.FS.quaternion, [gipper], ry.OT.sos, [1e3], target=[0, 0, 0.7068252, 0.7073883])
    komo.optimize()
    return komo

In [9]:
def move_to_object(gripper, obj):
    komo = C.komo_path(1.,1, tau, True) 
    komo.clearObjectives()
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
    komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
    komo.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter",t_object], ry.OT.eq, [1e2])
    komo.addObjective([1.], ry.FS.scalarProductXZ, [t_object, "R_gripperCenter"], ry.OT.eq, target=[1])
    komo.addObjective([1.], ry.FS.scalarProductYZ, ["R_gripperCenter", t_object], ry.OT.eq)
    komo.addObjective(time=[1.], feature=ry.FS.qItself, type=ry.OT.eq, order=1);
    komo.addObjective([], ry.FS.qItself, ["R_finger1"], ry.OT.eq, [1e1], order=1)
    komo.addObjective([], ry.FS.qItself, ["R_finger2"], ry.OT.eq, [1e1], order=1)
    komo.optimize()
    return komo



In [10]:
points = []
tau = .01
t_object = "tire"
closed = 0

target_pos = np.array([1.16, -0.75, 1])
for t in range(500):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()

    V.recopyMeshes(C)
    V.setConfiguration(C)
    
    p_obj = obj.getPosition()
    
    if not S.getGripperIsGrasping("R_gripper"):
        [y,J] = C.evalFeature(ry.distance, ["R_gripperCenter", t_object])
        #print(abs(y))
        if abs(y) < 0.04 and closed == 0:
            closed = 1
            print(S.getGripperWidth("R_gripper"))
            S.closeGripper("R_gripper")
            #t_object = "object1"
            print("graspping", t)
            print(S.getGripperWidth("R_gripper"))
            
    else:
        #print("while closing ",S.getGripperWidth("R_gripper"))
        [y,J] = C.evalFeature(ry.FS.position, ["R_gripperCenter"])
        distance = np.linalg.norm(y-target_pos)
        if distance < 0.05 and closed == 1:
            S.openGripper("R_gripper")
            print("opening")
            closed = 2
        # add a grasping condition
        # and utilize
        # S.closeGripper("R_gripper")

    # you can add your manipulation code here
    # Tips, not restricted: compute a one-step komo path
    # and compute the configuration q
    # komo = C.komo_path(1.,1,tau,True)

    if closed == 0:
        komo = move_to_object("R_gripperCenter", t_object)
    elif closed == 1 and S.getGripperIsGrasping("R_gripper"):
        komo = move_to_position("R_gripperCenter", target_pos)
    
    if closed != 2:
        C.setFrameState( komo.getConfiguration(0))
        #V.setConfiguration(C)
        q = C.getJointState()
    
    S.step(q, tau, ry.ControlMode.position)
    
    #S.closeGripper()

-5.659706821303137e-09
graspping 24
-5.659706821303137e-09
opening


In [18]:
[y,J] = C.evalFeature(ry.FS.position, ["R_gripperCenter"])
print(y)
distance = np.linalg.norm(y-target_pos)
print(distance)

[ 1.16000563 -0.75000968  0.99999634]
1.1779579148354401e-05


In [ ]:
----ccopied---

In [8]:
points = []
tau = .01
t_object = "tire"
closed = 0
for t in range(800):
    time.sleep(0.001)

    #grab sensor readings from the simulation
    q = S.get_q()

    V.recopyMeshes(C)
    V.setConfiguration(C)
    
    p_obj = obj.getPosition()
    
    if not S.getGripperIsGrasping("R_gripper"):
        [y,J] = C.evalFeature(ry.distance, ["R_gripperCenter", t_object])
        #print(abs(y))
        if abs(y) < 0.1 and closed == 0:
            closed = 1
            print(S.getGripperWidth("R_gripper"))
            S.closeGripper("R_gripper")
            #t_object = "object1"
            print("graspping", t)
            print(S.getGripperWidth("R_gripper"))
            
    else:
        #print("while closing ",S.getGripperWidth("R_gripper"))
        t_object = "L_gripperCenter"
        [y,J] = C.evalFeature(ry.distance, ["R_gripperCenter", t_object])
        if abs(y) < 0.2 and closed == 1:
            S.openGripper("R_gripper")
            print("opening")
            closed = 2
        # add a grasping condition
        # and utilize
        # S.closeGripper("R_gripper")

    # you can add your manipulation code here
    # Tips, not restricted: compute a one-step komo path
    # and compute the configuration q
    # komo = C.komo_path(1.,1,tau,True)

    if closed != 2 :
        komo = C.komo_path(1.,1, tau, True) 
        komo.clearObjectives()
        komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
        komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
        komo.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter",t_object], ry.OT.eq, [1e2])
        
        if closed == 0:
            komo.addObjective([1.], ry.FS.scalarProductXZ, [t_object, "R_gripperCenter"], ry.OT.eq, target=[1])
            komo.addObjective([1.], ry.FS.scalarProductYZ, ["R_gripperCenter", t_object], ry.OT.eq)

        komo.addObjective(time=[1.], feature=ry.FS.qItself, type=ry.OT.eq, order=1);
        komo.addObjective([], ry.FS.qItself, ["R_finger1"], ry.OT.eq, [1e1], order=1)
        komo.addObjective([], ry.FS.qItself, ["R_finger2"], ry.OT.eq, [1e1], order=1)
        komo.optimize()
        #komo.getReport()
        C.setFrameState( komo.getConfiguration(0))
        #V.setConfiguration(C)
        q = C.getJointState()  

    S.step(q, tau, ry.ControlMode.position)
    #S.closeGripper()

-1.2398753713582315e-14
graspping 15
-1.2398753713582315e-14


In [8]:
closed=1

In [4]:
points = []
tau = .01

for t in range(300):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
        
    S.step([], tau, ry.ControlMode.none)

In [5]:
#-- MODEL WORLD configuration, this is the data structure on which you represent
# what you know about the world and compute things (controls, contacts, etc)
C = ry.Config()
#D = C.view() #rather use the ConfiguratioViewer below
C.addFile("../../scenarios/pandasTable.g")

RuntimeError: /home/vijay/Desktop/practical-robotics/robotics-course/rai/rai/Core/util.cpp:cd_file:1143(-2) couldn't change to directory '../../scenarios' from '/home/vijay/Desktop/practical-robotics/robotics-course/frames_check'

In [6]:
#-- using the viewer, you can view configurations or paths
V = ry.ConfigurationViewer()
V.setConfiguration(C)

In [7]:
#-- the following is the simulation loop
tau = .01

for t in range(5):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q
    V.setConfiguration(C) #to update your model display
    [y,J] = C.evalFeature(ry.FS.position, ["R_gripper"])
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ [0.,0.,-1e-1];

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)

RuntimeError: /home/vijay/Desktop/practical-robotics/robotics-course/rai/rai/Kin/TM_default.cpp:phi:103(-2) CHECK failed: 'a' -- 

## doing things relative to an object

In [7]:
# add a new frame to the MODEL configuration
# (Perception will later have to do exactly this: add perceived objects to the model)
capsule = C.addFrame("capsule")
box     = C.addFrame("box")

In [8]:
# set frame parameters, associate a shape to the frame, 
#capsule.setPosition([.8,0,1.5])
#capsule.setPosition([0.1,-0.75,1.25])
#capsule.setQuaternion([1,0,.5,0])



In [103]:
Rect_Center = [-0.3,0.4,1.5]
Rect_Width  = 0.4
Rect_Height = 0.2
Rect_Start  = [(Rect_Center[0] - Rect_Width/2), Rect_Center[1]-0.1, (Rect_Center[2] + Rect_Height/2)]

In [104]:
#capsule.setPosition([0.3,0,0.8])
capsule.setPosition([-0.1,0.2,1.5])
capsule.setQuaternion([1,0,0,0])

capsule.setShape(ry.ST.capsule, [.2,.02])
capsule.setColor([1,0,1])

In [105]:
box.setPosition(Rect_Center)
box.setQuaternion([1,0,0,0])
box.setShape(ry.ST.box, [Rect_Width,0.05,Rect_Height])
box.setColor([1,1,0])

In [106]:
V.setConfiguration(C)
#V.recopyMeshes(C) #this is rarely necessary, only when you change meshes within C

In [9]:
def DetectCollision(C):
    collisionPresent = False
    coll = C.feature(ry.FS.accumulatedCollisions, [])
    C.computeCollisions()
    coll.eval(C)
    Collisions = C.getCollisions(0)
    if len(Collisions) > 0:
        for i in Collisions:
            if ('L_' in i[0] and 'R_' in i[1]) or ('R_' in i[0] and 'L_' in i[1]):
                print(i)
                collisionPresent = True
    return collisionPresent

In [108]:
state = 0

def DrawRectangle(Rect_Pos):
    global state
    
    if state == 0:
        Rect_Pos =  Rect_Start
    elif state == 1:
        Rect_Pos[0] += Rect_Width
    elif state == 2:
        Rect_Pos[2] -= Rect_Height
    elif state == 3:
        Rect_Pos[0] -= Rect_Width
    elif state == 4:
        Rect_Pos[2] += Rect_Height
        
    state = (state+1) % 5
    return Rect_Pos

In [109]:
iteration = 5000
tau = .01
y_box = np.zeros(3)
align = True
Rect_Pos = Rect_Start


[y_itself ,J_itself]  = C.evalFeature(ry.FS.position, ["L_lift"])
[y_GripInit ,J_GripInit]  = C.evalFeature(ry.FS.position, ["L_gripperCenter"])


#-- the following is the simulation loop
for t in range(iteration):
    y = []
    J = []
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q
    V.setConfiguration(C) #to update your model display
    
    # position difference between Right gripper and the capsule
    [y_capsule,J_capsule] = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter", "capsule"])
    
    # position difference between Left gripper and the box 
    #[y_box    ,J_box]     = C.evalFeature(ry.FS.positionDiff, ["L_gripperCenter", "capsule"])
    [y_GripCen    ,J_GripCen]     = C.evalFeature(ry.FS.position, ["L_gripperCenter"])
    
    #y_target = y_GripInit + ((t+1)/iteration) * (Rect_Center - y_GripInit)
    # Draw Rectangle
    
    if(np.linalg.norm(y_box)) < 5e-2:
        Rect_Pos = DrawRectangle(Rect_Pos)
    
    y_box = y_GripCen - Rect_Pos
    
        
    # Make Right gripper parallel to Y axis by making orthogonal to other 2 directions
    [y_capZZ, J_capsuleZZ] = C.evalFeature(ry.FS.scalarProductZZ, ["R_gripperCenter", "capsule"])
    [y_capXZ, J_capsuleXZ] = C.evalFeature(ry.FS.scalarProductXZ, ["R_gripperCenter", "capsule"])
    
    [y_LGrip  ,J_LGrip]   = C.evalFeature(ry.FS.position, ["L_lift"])
    
    # reach capsule
    y = y_capsule/tau
    J = J_capsule/tau
    
    
    if align == True:
        # make left gripper parallel to Rectangle boundaries
        [y_RectZZ, J_RectZZ] = C.evalFeature(ry.FS.scalarProductZZ, ["box", "L_gripperCenter"])
        [y_RectXZ, J_RectXZ] = C.evalFeature(ry.FS.scalarProductXZ, ["L_gripperCenter", "box"])
        
        # merge tasks for gripper handle parallel to rectangle
        y  = np.concatenate((y ,y_RectZZ/tau,y_RectXZ/tau))
        J = np.vstack((J, J_RectZZ/tau, J_RectXZ/tau))

    # stay close to initial position
    y  = np.concatenate((y ,  (y_LGrip - y_itself)))
    J = np.vstack((J,  J_LGrip))    

    # merge all tasks
    y  = np.concatenate((y , y_capZZ/tau, y_capXZ/tau, y_box/tau))
    J = np.vstack((J,  J_capsuleZZ/tau, J_capsuleXZ/tau, J_GripCen/tau))
    
    CollStatus = DetectCollision(C)
    
    if CollStatus == True:
        print("Possible Chance of Collision...Terminating")
        break
    
    # calculate joint velocities
    psudeoInverseJ = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0]))
    vel =  psudeoInverseJ @ (-y)
    
    # Directly stop Left gripper lifting
    vel[0] = 0
    
    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)
    


('L_panda_coll7', 'R_finger2', -0.00013082130226798813)
Possible Chance of Collision...Terminating


How could you align the gripper for a proper grasp?

In [111]:
S=0
V=0
C=0
RealWorld=0

## Direct inverse Kinematics with multiple objectives

In [ ]:
import os
os._exit(0)

In [1]:
import sys
sys.path.append('../../build')
import libry as ry
import numpy as np
import time

In [2]:
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/challenge.g")

S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

In [3]:
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")

V = ry.ConfigurationViewer()
V.setConfiguration(C)

In [5]:
tau = .01

for t in range(300):
    time.sleep(0.01)

    q = S.get_q()
    if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

    C.setJointState(q) #set your robot model to match the real q
    V.setConfiguration(C) #to update your model display

    #evaluate a first feature
    [y1,J1] = C.evalFeature(ry.FS.position, ["R_gripper"])
    #redefine y1 to become the desired change-of-value ("error" or "residual"); here just a constant velocity
    y1 = np.array([0.,0.,-1e-1])
    #you can multiply y1 and J1 here with some number, to increase the importance of the first feature
    
    #evaluate a second feature
    [y2,J2] = C.evalFeature(ry.FS.scalarProductYZ, ["R_gripper","world"])
    #redefine y2 to become the desired change-of-value ("error" or "residual"); here by subtracting the target
    y2 = [1.] - y2
    #you can multiply y2 and J2 here with some number, to increase the importance of the second feature

    #stack all tasks
    y = np.block([y1, y2])
    J = np.block([[J1],[J2]])
    
    vel = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0])) @ y;

    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)

KeyboardInterrupt: 

In [ ]:
#C.getJointNames()
Rect_Center

In [ ]:
# Null space motion 
#delA = np.ones(17) * 0.1
#delA[0]=delA[-1] = delA[8] = 0
#vel =  psudeoInverseJ @ (-y) + (np.eye(17) - psudeoInverseJ @ J) @ delA;

In [110]:
# Code with Left gripper picking up object
# currently not used

'''

#-- the following is the simulation loop
tau = .01

#[y_itself ,J_itself]  = C.evalFeature(ry.FS.position, ["L_gripper"])

for t in range(500):
    y = []
    J = []
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
            [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow

    #some good old fashioned IK
    C.setJointState(q) #set your robot model to match the real q
    V.setConfiguration(C) #to update your model display
    
    [y_capsule,J_capsule] = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter", "capsule"])
    #[y_box    ,J_box]     = C.evalFeature(ry.FS.positionDiff, ["L_gripperCenter", "box"])
    #[y_LGrip  ,J_LGrip]   = C.evalFeature(ry.FS.position, ["L_gripper"])

    [y_capZZ, J_capsuleZZ] = C.evalFeature(ry.FS.scalarProductZZ, ["R_gripperCenter", "capsule"])
    [y_capXZ, J_capsuleXZ] = C.evalFeature(ry.FS.scalarProductXZ, ["R_gripperCenter", "capsule"])
    
    # merge all tasks
    
    # reach capsule
    y = y_capsule/tau
    J = J_capsule/tau
    
    # stay close to initial position
    #y  = np.concatenate((y ,  (y_LGrip - y_itself)))
    #J = np.vstack((J,  J_LGrip))    
    
    # reach box
    y  = np.concatenate((y , y_capZZ/tau, y_capXZ/tau, y_box/tau))
    J = np.vstack((J,  J_capsuleZZ/tau, J_capsuleXZ/tau, J_box/tau))
    
    psudeoInverseJ = J.T @ np.linalg.inv(J@J.T + 1e-2*np.eye(y.shape[0]))
    
    vel =  psudeoInverseJ @ (-y)
    
    # stop Left gripper lifting
    vel[0] = 0
    
    #send velocity controls to the simulation
    S.step(vel, tau, ry.ControlMode.velocity)
'''


'\n\n#-- the following is the simulation loop\ntau = .01\n\n#[y_itself ,J_itself]  = C.evalFeature(ry.FS.position, ["L_gripper"])\n\nfor t in range(500):\n    y = []\n    J = []\n    time.sleep(0.01)\n\n    #grab sensor readings from the simulation\n    q = S.get_q()\n    if t%10 == 0:\n            [rgb, depth] = S.getImageAndDepth()  #we don\'t need images with 100Hz, rendering is slow\n\n    #some good old fashioned IK\n    C.setJointState(q) #set your robot model to match the real q\n    V.setConfiguration(C) #to update your model display\n    \n    [y_capsule,J_capsule] = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter", "capsule"])\n    #[y_box    ,J_box]     = C.evalFeature(ry.FS.positionDiff, ["L_gripperCenter", "box"])\n    #[y_LGrip  ,J_LGrip]   = C.evalFeature(ry.FS.position, ["L_gripper"])\n\n    [y_capZZ, J_capsuleZZ] = C.evalFeature(ry.FS.scalarProductZZ, ["R_gripperCenter", "capsule"])\n    [y_capXZ, J_capsuleXZ] = C.evalFeature(ry.FS.scalarProductXZ, ["R_gripperCente